# Template Matching

## 1.0 Import Packages

In [47]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## 2.0 Read Pickle File

In [48]:
def readpickefile(person):
    with open(person+'.pkl', 'rb') as f:
        result1,dfs1 = pickle.load(f)
    return result1,dfs1

result1,dfs1 = readpickefile('person1')
result2,dfs2 = readpickefile('person2')
result3,dfs3 = readpickefile('person3')
result4,dfs4 = readpickefile('person4')
result5,dfs5 = readpickefile('person5')

## 3.0 Segment out distance > 10m or < 3m

In [49]:
def remove_10_3(rotate,angle):
    rot1_3000_10000 = rotate[angle["DST"]>3000]
    rot1_3000_10000 = rot1_3000_10000[angle["DST"]<10000]
    ang1_3000_10000 = angle[angle["DST"]>3000]
    ang1_3000_10000 = ang1_3000_10000[angle["DST"]<10000]
    return rot1_3000_10000, ang1_3000_10000

## 4.0 Split Data to 80:20

In [50]:
def splitdata80_20(dfs1):
    dfs_list1=list(dfs1.keys())
    dfs_rot_train=[]
    dfs_rot_test=[]
    for ind, rsbag1 in enumerate(dfs_list1):
        rot1, ang1 = dfs1[rsbag1][0]
        rot1, ang1 = remove_10_3(rot1, ang1)
        if ind<8:
            dfs_rot_train.append(rot1)
        else:
            dfs_rot_test.append(rot1)    
    return dfs_rot_train, dfs_rot_test

dfs_rot_train1, dfs_rot_test1=splitdata80_20(dfs1)
dfs_rot_train2, dfs_rot_test2=splitdata80_20(dfs2)
dfs_rot_train3, dfs_rot_test3=splitdata80_20(dfs3)
dfs_rot_train4, dfs_rot_test4=splitdata80_20(dfs4)
dfs_rot_train5, dfs_rot_test5=splitdata80_20(dfs5)

## 5.0 Test Accuracy with Test Dataset using template matching correlation

In [51]:
def correlation_features_rotate_compare_one_other(data,dfs_rot_train):
    correlation=[]
    for rot1 in dfs_rot_train:
        #Compute Own first
        column_name=list(rot1.columns)
        for main_c,sub_c in column_name:
            test1=data[main_c][sub_c]
            #Q1 = test1.quantile(0.25)
            #Q3 = test1.quantile(0.75)
            #IQR = Q3 - Q1
            #test1=test1[test1 > (Q1 - 1.5 * IQR)]
            #test1=test1[test1 < (Q3 + 1.5 * IQR)]
            a=pd.Series(test1)
            new1=a.interpolate(method='polynomial', order=2)
            new12= list(new1)
            
            test2=rot1[main_c][sub_c]
            #Q1 = test2.quantile(0.25)
            #Q3 = test2.quantile(0.75)
            #IQR = Q3 - Q1
            #test2=test2[test2 > (Q1 - 1.5 * IQR)]
            #test2=test2[test2 < (Q3 + 1.5 * IQR)]
            a=pd.Series(test2)
            new2=a.interpolate(method='polynomial', order=2)
            new22= list(new2)
            new122=pd.Series(new12)
            new222=pd.Series(new22)
            result=new122.corr(new222)
            correlation.append(abs(result))
    result = np.reshape(correlation, (-1,36))
    df = pd.DataFrame(result, columns = column_name)
    final_correlation=statistics.mean(df.mean(axis = 0, skipna = True))
    return final_correlation

def calculate_accuracy(dfs_rot_test1,truth):
    correct = 0
    for test in dfs_rot_test1:
        correlation1 = correlation_features_rotate_compare_one_other(test,dfs_rot_train1)
        correlation2 = correlation_features_rotate_compare_one_other(test,dfs_rot_train2)
        correlation3 = correlation_features_rotate_compare_one_other(test,dfs_rot_train3)
        correlation4 = correlation_features_rotate_compare_one_other(test,dfs_rot_train4)
        correlation5 = correlation_features_rotate_compare_one_other(test,dfs_rot_train5)
        list_correlation = [correlation1,correlation2,correlation3,correlation4,correlation5]
        ans = list_correlation.index(max(list_correlation))
        if ans==truth:
            correct = correct +1
    return correct
            
correct1=calculate_accuracy(dfs_rot_test1,0)
correct2=calculate_accuracy(dfs_rot_test2,1)
correct3=calculate_accuracy(dfs_rot_test3,2)
correct4=calculate_accuracy(dfs_rot_test4,3)
correct5=calculate_accuracy(dfs_rot_test5,4)
accuracy = (correct1+correct2+correct3+correct4+correct5)/(len(dfs_rot_test1)+len(dfs_rot_test2)+len(dfs_rot_test3)+len(dfs_rot_test4)+len(dfs_rot_test5))

In [52]:
print(accuracy)

0.36363636363636365
